In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib tk

import os 
import time 
import pianoq
import datetime
from TC300_COMMAND_LIB import TC300
from pianoq.lab.asi_cam import ASICam
from pianoq.lab.photon_counter import PhotonCounter
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.photon_scan import PhotonScanner

can't import TimeTagger


In [ ]:
tc = TC300()
serial = tc.list_devices()[0][0]
tc.open(serial, 115200, 3)
tc.set_target_temperature(2, 75)

In [ ]:
tc.close()

In [ ]:
cam = ASICam()
base_path = r'G:\My Drive\People\Ronen\PHD\SPDC2025\phase_matching'
cam.set_exposure(1)

In [ ]:
cam.close()

## Find phase matching temperature

In [ ]:
t = 0
while t < 72:
    time.sleep(2)
    tt = [0]
    tc.get_actual_temperature(2, tt)
    t = round(tt[0], 4)
    cam.save_image(os.path.join(base_path, f"T={t}.fits"))

## Check photon counter 

In [3]:
ph = PhotonCounter(serial_port='COM8')

In [ ]:
while True:
    x = ph.read()
    print(x[0])

In [16]:
ph.close()

## Photon scan 

In [2]:
x_motor = ThorlabsKcubeDC(serial_number=27253522)
y_motor = ThorlabsKcubeStepper(serial_number=26001271)
ph = PhotonCounter(serial_port='COM8')

In [3]:
x_motor.move_relative(0.1)
y_motor.move_relative(-0.1)

In [4]:
print(x_motor.get_position())
print(y_motor.get_position())

0.8001
0.899999


In [5]:
integration_time = 1 
start_x = 0.4
start_y = 0.2 
x_pixels = 5
y_pixels = 5
pixel_size_x = 0.1
pixel_size_y = 0.1
# name = 'find_image'
name = 'test'

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
dir_path = r'G:\My Drive\People\Ronen\PHD\SPDC2025\phase_matching\Scans'

path = f'{dir_path}\\{timestamp}_{name}.scan'
scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y,
                        run_name=name, is_timetagger=False, coin_window=2e-9, saveto_path=path)

In [ ]:
# TODO: For some reason this gets stuck after a few positions. Need to figure out what the problem is. 
single1s, single2s, coincidences = scanner.scan(x_motor=x_motor, y_motor=y_motor, ph=ph)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 0. Singles2: 0. Coincidence: 0.00.


In [ ]:
x_motor.close()
y_motor.close()  # pesky bug?
tt.close()